In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
max_seq_length = 2048 # Supports RoPE Scaling interally, so choose any!

In [ ]:
alpaca_prompt = """Di bawah ini adalah instruksi yang menggambarkan sebuah tugas dengan input yang memberikan konteks lebih lanjut. Tuliskan respons yang tepat untuk menyelesaikan permintaan tersebut.

### Instruksi:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Aryaduta/modellora9", # or Change to your own model from HuggingFace
    max_seq_length = max_seq_length,
    dtype = None,
)

In [ ]:
full_instruction = """
Objektif: Tugas anda adalah menghasilkan urutan respons JSON untuk merencanakan tindakan lengan robot berdasarkan input pengguna. Jika tujuan tidak dapat dicapai dengan menggunakan instruksi yang disediakan dan objek yang tersedia, kembalikan pesan kesalahan.
    
Berikan objek JSON yang mengandung array "actions", diidentifikasi dengan key "actions".
    
Setiap aksi harus direpresentasikan sebagai objek dengan "command" dan "parameters" yang sesuai
    
Objek dan Koordinat yang Tersedia:
{object_context}
    
Perintah yang Tersedia:
1. move: Gerakkan lengan robot ke arah tertentu. Sertakan parameter "direction" dengan nilai "atas", "bawah", "depan", "belakang", "kiri", atau "kanan".
2. move_to: Gerakkan lengan robot ke koordinat tertentu. Sertakan parameter "x", "y", dan "z" untuk menentukan koordinat tujuan.
3. suction_cup: Aktifkan atau nonaktifkan cup hisap. Gunakan parameter "action" dengan nilai "on" atau "off".
5. err_msg: Kembalikan pesan kesalahan jika tujuan pengguna tidak dapat tercapai dengan menggunakan objek dan perintah saat ini. Gunakan parameter "msg" dengan nilai "tidak dapat membuat rencana aksi dengan kondisi terkini".
    
Contoh Penggunaan Perintah:
{{"actions":[{{"command":"move","parameters":{{"direction":"up"}}}},{{"command":"move_to","parameters":{{"x":-265.56,"y":196.34,"z":-50}}}},{{"command":"suction_cup","parameters":{{"action":"on"}}}},{{"command":"err_msg","parameters":{{"msg":"tidak dapat membuat rencana aksi dengan kondisi terkini"}}}}]}}
    
Instruksi Penggunaan:
1. Untuk memindahkan objek yang tersedia ke koordinat tertentu, aktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "on", kemudian gerakkan ke koordinat objek menggunakan perintah "move_to".
2. Berikan koordinat penempatan untuk tujuan pengguna menggunakan perintah "move_to" sesuai dengan koordinat yang diberikan pengguna atau koordinat objek yang disediakan. Jangan lupa untuk menambahkan tanda negati
3. Untuk melepaskan objek setelah menggunakan penyedot, nonaktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "off".
4. Untuk memindahkan robot secara lateral (misalnya, ke kiri atau kanan), gunakan perintah "move" dengan arah yang sesuai.
5. Untuk gerakan lateral objek (kiri atau kanan), pertama-tama gerakkan lengan robot ke koordinat objek menggunakan perintah "move_to", kemudian gunakan perintah "move" dengan arah yang sesuai.
6. Jika tujuan pengguna tidak dapat tercapai dengan perintah dan objek saat ini, gunakan perintah "err_msg".
"""

def inferrence(input_context, object_context):
    # Enable native 2x faster inference
    FastLanguageModel.for_inference(model)

    full_instruction2 = full_instruction.format(object_context=object_context)

    
    # Prepare inputs
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                full_instruction2,
            input_context,  # input
            "",  # output - leave this blank for generation!
        )
        ],
        return_tensors="pt"
    ).to("cuda")
    
    # Generate outputs
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    
    # Decode output and extract JSON response
    decoded_output = tokenizer.batch_decode(outputs)[0]
    
    # Find the start and end indices for the JSON response
    start_marker = "### Response:"
    end_marker = "<eos>"
    start_index = decoded_output.find(start_marker) + len(start_marker)
    end_index = decoded_output.find(end_marker, start_index)
    
    # Extract the JSON response
    json_response = decoded_output[start_index:end_index].strip()
    
    # Print the JSON response
    print(json_response)

    # return the JSON response
    return json_response

In [ ]:
object_context = """ 
1. Balok ungu = (-86.59, 117.21, -122.30)
2. Balok kuning = (-168.94, -129.37, -68)
3. Balok biru = (152.76, 158.92, 6)
"""

input_context = "geser balok biru ke kanan"

inferrence(input_context, object_context)